<a href="https://colab.research.google.com/github/HpDiniz/Analise-Financeira/blob/main/Projeto_de_Pesquisa_Mestrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings; 
warnings.simplefilter('ignore')

# 0. Install and Import Dependencies

In [4]:
!pip install pystan
!pip install statsmodels
!pip install yahoo-finance
!pip install yfinance
!pip install xgboost
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 7.0 MB/s 
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7233 sha256=036bb284e806f30e06ac04d3e6f376381bf634672ab1302d837998d1122234ac
  Stored in directory: /root/.cache/pip/wheels/74/3c/6e/504622cec9991612b9e3945fe8b375fef727854b42eb333c91
Successfully built yahoo-finance
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 18.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successf

In [5]:
import re
import bs4
import json
import pickle
import requests
import datetime
import dateutil

from datetime import date
from bs4 import BeautifulSoup
from xgboost import XGBRegressor
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX

import numpy as np
import pandas as pd
import yfinance as yf

# 1. Read in Data and Process Dates

In [6]:
first_day = pd.to_datetime('today').replace(day=1,hour=0,minute=0,second=0,microsecond=0)
this_month = (first_day).strftime("%Y-%m")
last_month = (first_day - relativedelta(months=1)).strftime("%Y-%m")

headers = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}

In [7]:
def updateColumn(columnName, line, newRow):
    if '<b>'+columnName+':</b>' in line:
        item = line.replace('<b>'+columnName+':</b>',"").replace("<li>","")
        item = item.replace("</li>","").replace("m<sup>2</sup>","")
        item = item.replace("N/A","").strip()
        newRow[columnName] = item

def converteData(datas, monthYearOnly):

    new_array = []
    meses = ["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]

    for data in datas:

        item = data.split("/")
        mes = str(meses.index(item[0])+1)
        mes = ("0" + mes)[len(mes)-1:len(mes)+1]

        new_date = item[1] + "-" + mes

        if not monthYearOnly:
            new_date = new_date + "-01 00:00:00"
        
        new_array.append(new_date)
        
    return new_array

def xgboostPrediction(dataframe, column, month):

    new_df = dataframe.copy()

    predict_column = column + "_target"

    new_df[predict_column] = new_df[column].shift(-month)

    treino = new_df[:-1]
    validacao = new_df[-1:]

    if month > 1:
        treino = new_df[:-month]
        validacao = new_df[-month:-(month-1)]

    x_treino = treino.loc[:, [column]].values
    y_treino = treino.loc[:, [predict_column]].values

    modelo_xgb = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    modelo_xgb.fit(x_treino, y_treino)

    value = float(validacao[column][-1])
    predicao = modelo_xgb.predict(value)

    return (predicao[0])

def sarimaxPrediction(dataframe, columns, months):

    result_df = None

    for col in columns:
        arima_df_aux = dataframe[[col]]
        fit_arima = auto_arima(arima_df_aux, d=1, start_p=1, start_q=1, max_p=3, mar_q=3, seasonal=True, m=6, D=1, start_P=1,start_Q=1, max_P=2, max_Q=2, information_criterion='aic', trace=False, error_action='ignore', stepwise=True)

        model=SARIMAX(arima_df_aux, order=fit_arima.order, seasonal_order=fit_arima.seasonal_order)
        resultado_sarimax = model.fit()

        forecast = resultado_sarimax.get_forecast(steps=months)

        if result_df is None:
            result_df = pd.DataFrame.from_dict(forecast.predicted_mean)

        result_df[col] = forecast.predicted_mean

    return (result_df[columns])

def traditionalPredict(dfs, ativo, columns, months):

    dataframe = dfs[ativo]

    result_df = sarimaxPrediction(dataframe, columns, months)

    result_df['DividendYield'] = float(result_df['Dividends']/result_df['Close'])*100

    result_df['Ticker'] = ativo
    result_df['Prediction'] = True

    return result_df

def machineLearningPredict(dfs, ativo, columns, months_to_predict):

    xgboost_df = None

    dataframe = dfs[ativo]

    for m in range(len(months_to_predict)):

        predict_months = m+1

        result = {}

        for col in columns:
            result[col] = xgboostPrediction(dataframe,col, predict_months)

        result['DividendYield'] = float(result['Dividends']/result['Close'])*100

        result['Ticker'] = ativo
        result['Date'] = months_to_predict[m]
        result['Prediction'] = True

        result_df = pd.DataFrame.from_dict([result])

        result_df = result_df.set_index('Date')
        result_df.index = pd.to_datetime(result_df.index)

        if xgboost_df is None:
            xgboost_df = result_df.copy()
        else:
            xgboost_df = pd.concat([xgboost_df, result_df])

    return xgboost_df

In [8]:
def get_funds():

    headers = {
        'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
            ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }

    url = 'https://www.fundsexplorer.com.br/ranking'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]

    idx = df[df['Setor'].isna()].index
    df.drop(idx, inplace=True)

    df_funds = df.rename(columns={'Códigodo fundo': 'Ticker'})

    col_categorical = ['Ticker','Setor']
    df_funds[col_categorical] = df_funds[col_categorical].astype('category')

    df_funds.sort_values('Ticker', inplace=True)

    df_funds[col_categorical]

    return df_funds

def run_crawler():

    df_funds = get_funds()

    columns = {'Código': [],'Endereço': [], 'Bairro': [], 'Cidade': [], 'Área Bruta Locável': []}
    df_ativos = pd.DataFrame(columns)

    for fundo in dfs:

        response = requests.get(url_fundos + fundo, headers=headers)

        soup = bs4.BeautifulSoup(response.content, "html")
        div = soup.find("div", {"id": "dividends-chart-wrapper"})

        labels = re.findall('"labels":\[.*?\]', str(div))
        dividends = re.findall('"data":\[.*?\]', str(div))

        # parse data:
        dividends = json.loads("{" + dividends[0] + "}")['data']
        labels = json.loads("{" + labels[0] + "}")['labels']

        # converte "Março/2021" para "2021-03-01"
        datas = converteData(labels, False)

        for i in range(len(datas)):
            dfs[fundo].loc[dfs[fundo].index[dfs[fundo].index == datas[i]],'Dividends'] = dividends[i]
        
        #dfs[fundo]['Datetime'] = dfs[fundo].index
        #dfs[fundo]['DividendYield'] = float(dfs[fundo]['Dividends'][0]/dfs[fundo]['Close'][0]) * 100

        if ('Ativos do ') in str(response.content):

            print("Coletando ativos de " + fundo + "...")

            soup = BeautifulSoup(response.content,"lxml")
            w3schollsList = soup.find("div",id="fund-actives-items")

            lista = w3schollsList.find_all('ul')

            for l in lista:

                newRow = {}

                itemList = l.find_all('li')
                for it in itemList:
                    line = str(it)

                    for column in ['Endereço', 'Bairro', 'Cidade', 'Área Bruta Locável']:
                        updateColumn(column, line, newRow)

                if len(newRow) > 0:
                    newRow['Código'] = fundo
                    df_ativos = df_ativos.append(newRow, ignore_index = True)
        else:
            print(fundo + " não possui ativos.")

    columns_str = ['Código','Endereço', 'Bairro', 'Cidade']
    df_ativos[columns_str] = df_ativos[columns_str].astype("string")
    df_ativos['UF'] = df_ativos['Cidade'].str[-2:]
    df_ativos['Área Bruta Locável'] = pd.to_numeric(df_ativos['Área Bruta Locável'].str.replace(r"\.", "").str.replace(r"\,", "."), errors='coerce').astype(float)

    df_history = None
    for t in df['Códigodo fundo']:

        df_aux = dfs[t][['Datetime','Close','Dividends','DividendYield','Ticker','Prediction']]

        if df_history is None:
            df_history = df_aux.copy()
        else:
            df_history = df_history.append(df_aux, ignore_index = False)

    return df, df_ativos, df_history

def run_predictor(dfs, qnt_months_to_predict = 2):
    # Calculando datas de interesse
    months_to_predict = []

    for i in range(qnt_months_to_predict):
        add_month = dateutil.relativedelta.relativedelta(months=i)
        months_to_predict.append((first_day + add_month).strftime("%Y-%m-%d"))

    xgboost_dfs = None
    for t in dfs.keys():
        print("Predicting with Machine Learning: " + t + "...")
        aux_df = machineLearningPredict(dfs,t,['Close','Dividends'],months_to_predict)

        if xgboost_dfs is None:
            xgboost_dfs = aux_df.copy()
        else:
            xgboost_dfs = pd.concat([xgboost_dfs, aux_df])

    df_history = None
    xgboost_dfs['Datetime'] = xgboost_dfs.index

    for t in dfs.keys():

        dfs[t]['Datetime'] = dfs[t].index
        dfs[t]['DividendYield'] = float(dfs[t]['Dividends'][0]/dfs[t]['Close'][0]) * 100

        df_aux = dfs[t][['Datetime','Close','Dividends','DividendYield','Ticker','Prediction']]

        if df_history is None:
            df_history = df_aux.copy()
        else:
            df_history = df_history.append(df_aux, ignore_index = False)

        df_history = df_history.append(xgboost_dfs[(xgboost_dfs['Ticker'] == t)].copy(), ignore_index = False)

    return df_history

In [ ]:
df, df_ativos, df_history = run_crawler()

NameError: ignored

In [ ]:
df_history[df_history['Ticker'] == 'XTED11']

In [ ]:
df_history = run_predictor(dfs, 2)

In [ ]:
df_history[df_history['Ticker'] == 'HGLG11']

In [ ]:
dfs['XTED11']

# 2. Data Visualization

In [340]:
def get_all_funds():

    response = requests.get('https://www.fundsexplorer.com.br/ranking', headers=headers)
    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]

    idx = df[df['Setor'].isna()].index
    df.drop(idx, inplace=True)

    df_funds = df.rename(columns={'Códigodo fundo': 'Ticker'})

    col_categorical = ['Ticker','Setor']
    df_funds[col_categorical] = df_funds[col_categorical].astype('category')

    df_funds.sort_values('Ticker', inplace=True)

    df_funds = df_funds[['Ticker','Setor','QuantidadeAtivos']].reset_index(drop=True)

    return df_funds

def get_close(fund, years):

    df_close = pd.DataFrame()

    end_date = (first_day).strftime("%d-%m-%Y")
    start_date = (first_day - relativedelta(months=years*12)).strftime("%d-%m-%Y")

    response = requests.get('https://fii-api.infomoney.com.br/api/v1/fii/cotacao/historico/grafico?Ticker='+fund+'&DataInicio='+start_date+'&DataFim='+end_date, headers=headers)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        if 'errors' in json_response:
            print(str(json_response['errors']))
        else:
            df_close = pd.read_json(json.dumps(json_response['dataValor']))
            df_close['Ticker'] = fund
            df_close.rename(columns={'valor': 'Close'}, inplace = True)
            df_close['Datetime'] = pd.to_datetime(df_close['data']).dt.strftime('%Y-%m-%d')
            df_close.drop(columns={'data'}, inplace = True)
        
    return df_close

def get_dividends(fund, years):

    start_date = (first_day - relativedelta(months=years*12)).strftime("%Y-%m-%d")

    df_dividends = pd.DataFrame()

    response = requests.get('https://fii-api.infomoney.com.br/api/v1/fii/provento/historico?Ticker='+fund, headers=headers)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        if "errorMessages" in json_response:
            print(json_response["errorMessages"])
        else:
            df_dividends = pd.read_json(response.content)

            if len(df_dividends) > 0:

                df_dividends.rename(columns={'ticker': 'Ticker', 'rendimento': 'Dividends', 'yield': 'DividendYield'}, inplace = True)
                df_dividends['Datetime'] = pd.to_datetime(df_dividends['data']).dt.strftime('%Y-%m-%d')

                df_dividends.drop(columns={'cnpj','data'}, inplace = True)
                df_dividends = df_dividends[df_dividends['Datetime'] >= start_date]

    return df_dividends

def get_history(fund, years):

    df_close = get_close(fund, years)
    df_dividends = get_dividends(fund, years)

    df_history = df_dividends.copy()

    if len(df_history) > 0 and len(df_close) > 0:

        df_close = df_close[df_close['Datetime'] >= df_dividends['Datetime'].min()]
        df_dividends = df_dividends[df_dividends['Datetime'] >= df_close['Datetime'].min()]
        
        new_df = []
        for index, row in df_history.iterrows():
            #print("Procurando 'Close' de: " + row['Datetime'])
            row['Dividends'] = round(row['Dividends'],2)
            row['Close'] = round(df_close[df_close['Datetime'] == row['Datetime']]['Close'].values[0],2)
            row['DividendYield'] = round(100*row['Dividends']/row['Close'],2)
            new_df.append(row)

        df_history = pd.DataFrame(new_df)
        
        df_history['Predicted'] = False
        df_history['Predicted'] = df_history['Predicted'].astype('bool')

    return df_history

def run_crawler(years):

    # Obtém a lista de todos os fundos imobiliários existentes
    df_funds = get_all_funds()

    df_history = pd.DataFrame()

    # Percorre a lista de fundos para obter o histórico individual de cada um deles
    for fund in df_funds['Ticker']:

        print("Coletando informações de " + fund + "...")

        #df_adress = get_history('ALZR11') #get_adress(response, fund)

        df_aux = get_history(fund, years)
        
        df_history = df_history.append(df_aux)

        print(str(len(df_aux)) + " dados de histórico")# e " + str(len(df_adress)) + " endereços foram encontrados.")

    return df_funds, df_history#, df_adress

In [341]:
# df_funds, df_history, df_adress = run_crawler()
df_funds, df_history = run_crawler(3)
#df_funds
#print(df_history)

Coletando informações de ABCP11...
22 dados de histórico
Coletando informações de AFHI11...
13 dados de histórico
Coletando informações de AFOF11...
11 dados de histórico
Coletando informações de AIEC11...
20 dados de histórico
Coletando informações de ALMI11...
0 dados de histórico
Coletando informações de ALZR11...
25 dados de histórico
Coletando informações de APTO11...
6 dados de histórico
Coletando informações de ARCT11...


IndexError: ignored

In [352]:
fund = 'ARCT11'
years = 3

df_close = get_close(fund, years)
df_dividends = get_dividends(fund, years)
df_dividends

,Ticker,Dividends,DividendYield,Datetime
0,ARCT11,10.140000,0.930275,2020-05-29
1,ARCT11,19.410000,1.941000,2020-06-30
2,ARCT11,10.860000,1.086000,2020-08-14
3,ARCT11,20.000000,1.869211,2020-09-15
4,ARCT11,10.860000,0.947627,2020-10-15
5,ARCT11,12.430000,1.090370,2020-11-16
6,ARCT11,29.147736,2.243842,2020-12-14
7,ARCT11,8.776183,0.680319,2021-01-15
8,ARCT11,5.368233,3.616189,2021-12-02
9,ARCT11,0.981845,0.701318,2021-12-03


In [350]:
df_history = df_dividends.copy()

if len(df_history) > 0 and len(df_close) > 0:

    df_close = df_close[df_close['Datetime'] >= df_dividends['Datetime'].min()]
    df_dividends = df_dividends[df_dividends['Datetime'] >= df_close['Datetime'].min()]

    new_df = []
    for index, row in df_history.iterrows():
        #print("Procurando 'Close' de: " + row['Datetime'])
        row['Dividends'] = round(row['Dividends'],2)
        row['Close'] = round(df_close[df_close['Datetime'] == row['Datetime']]['Close'].values[0],2)
        row['DividendYield'] = round(100*row['Dividends']/row['Close'],2)
        new_df.append(row)

    df_history = pd.DataFrame(new_df)

    df_history['Predicted'] = False
    df_history['Predicted'] = df_history['Predicted'].astype('bool')

df_history 

IndexError: ignored